In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/M2Datascale/Projet_Trans/BenchmarkLLM'

/content/drive/MyDrive/M2Datascale/Projet_Trans/BenchmarkLLM


In [4]:
import time
import pandas as pd

In [5]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 3.0 MB/s eta 0:00:00


In [6]:
pip show transformers

Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [7]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [10]:
data = pd.read_csv("./data.csv")

,Unnamed: 0,Question,Answers
0,58,What are the symptoms of Parkinson's Disease?,Treatment includes medications (levodopa) and ...
1,117,How is HIV/AIDS diagnosed?,Treatment includes antiretroviral therapy (ART...
2,166,How can I manage my Dengue Fever?,Diagnosis involves blood tests to detect the v...
3,278,Can Glaucoma be prevented?,Preventing Glaucoma involves regular eye exams...
4,261,Can Endometriosis be prevented?,Endometriosis is caused by the abnormal growth...
5,205,How can I manage my Multiple Sclerosis?,"Parkinson's Disease symptoms include tremors, ..."
6,135,How is Cancer diagnosed?,Symptoms vary depending on the type and stage ...
7,260,What treatment options are available for Endom...,Managing Endometriosis involves pain managemen...
8,5,How can I manage my Diabetes?,"Medication adherence, blood sugar monitoring."
9,162,How is Dengue Fever diagnosed?,"Lifestyle changes include avoiding alcohol, pr..."


In [15]:
def falconQA(question,reponse_attendue):
  started_time = time.time()
  sequences = pipeline(
      question,
      max_length=200,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  for seq in sequences:
    response = seq['generated_text']
  execution_time = time.time() - started_time
  new_row={'Model':'Falcon-7b', 'Questions':question, 'Reponses':response, 'Temps':execution_time, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [16]:
list_data_falcon=[]
for i in range(len(data)):
   list_data_falcon.append(falconQA(data.iloc[i]['Question'],data.iloc[i]['Answers']))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [17]:
list_data_falcon

[{'Model': 'Falcon-7b',
  'Questions': "What are the symptoms of Parkinson's Disease?",
  'Reponses': "What are the symptoms of Parkinson's Disease?\nSymptoms of Parkinson's Disease include tremors, slow movement, stiffness, and problems with coordination and balance. Other symptoms may include difficulty swallowing, changes in speech patterns, and a decrease in facial expressions.",
  'Temps': 3.680781364440918,
  'Reponses_Attendues': 'Treatment includes medications (levodopa) and deep brain stimulation.'},
 {'Model': 'Falcon-7b',
  'Questions': 'How is HIV/AIDS diagnosed?',
  'Reponses': 'How is HIV/AIDS diagnosed?\nHIV/AIDS can be diagnosed using a variety of tests, including HIV antibody tests, HIV DNA tests, and rapid HIV tests. The most accurate test is a combination of several tests, which can detect the virus at different stages of infection.',
  'Temps': 4.535125970840454,
  'Reponses_Attendues': 'Treatment includes antiretroviral therapy (ART) to control the virus.'},
 {'Mod

In [19]:
df_falcon = pd.DataFrame(data=list_data_falcon, index=None)

In [20]:
df_falcon

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Falcon-7b,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,3.680781,Treatment includes medications (levodopa) and ...
1,Falcon-7b,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nHIV/AIDS can be di...,4.535126,Treatment includes antiretroviral therapy (ART...
2,Falcon-7b,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nDengue feve...,8.151850,Diagnosis involves blood tests to detect the v...
3,Falcon-7b,Can Glaucoma be prevented?,Can Glaucoma be prevented?\nWhile there is no ...,2.783291,Preventing Glaucoma involves regular eye exams...
4,Falcon-7b,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nIt is difficu...,16.232186,Endometriosis is caused by the abnormal growth...
5,Falcon-7b,How can I manage my Multiple Sclerosis?,How can I manage my Multiple Sclerosis?\nThere...,5.455253,"Parkinson's Disease symptoms include tremors, ..."
6,Falcon-7b,How is Cancer diagnosed?,How is Cancer diagnosed?\nCancer is usually di...,4.216636,Symptoms vary depending on the type and stage ...
7,Falcon-7b,What treatment options are available for Endom...,What treatment options are available for Endom...,4.635307,Managing Endometriosis involves pain managemen...
8,Falcon-7b,How can I manage my Diabetes?,How can I manage my Diabetes?\nTo manage diabe...,4.537088,"Medication adherence, blood sugar monitoring."
9,Falcon-7b,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nDengue fever c...,4.925387,"Lifestyle changes include avoiding alcohol, pr..."


In [21]:
df_falcon.to_csv("data_falcon.csv", index=None)

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Falcon-7b,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,3.680781,Treatment includes medications (levodopa) and ...
1,Falcon-7b,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nHIV/AIDS can be di...,4.535126,Treatment includes antiretroviral therapy (ART...
2,Falcon-7b,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nDengue feve...,8.151850,Diagnosis involves blood tests to detect the v...
3,Falcon-7b,Can Glaucoma be prevented?,Can Glaucoma be prevented?\nWhile there is no ...,2.783291,Preventing Glaucoma involves regular eye exams...
4,Falcon-7b,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nIt is difficu...,16.232186,Endometriosis is caused by the abnormal growth...
5,Falcon-7b,How can I manage my Multiple Sclerosis?,How can I manage my Multiple Sclerosis?\nThere...,5.455253,"Parkinson's Disease symptoms include tremors, ..."
6,Falcon-7b,How is Cancer diagnosed?,How is Cancer diagnosed?\nCancer is usually di...,4.216636,Symptoms vary depending on the type and stage ...
7,Falcon-7b,What treatment options are available for Endom...,What treatment options are available for Endom...,4.635307,Managing Endometriosis involves pain managemen...
8,Falcon-7b,How can I manage my Diabetes?,How can I manage my Diabetes?\nTo manage diabe...,4.537088,"Medication adherence, blood sugar monitoring."
9,Falcon-7b,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nDengue fever c...,4.925387,"Lifestyle changes include avoiding alcohol, pr..."
